In [1]:
from pycaret.regression import *

In [2]:
import pandas as pd

In [3]:
train = pd.read_parquet('C:/Users/user/Desktop/bj/Bbeojung.kr/Data/final_dataset/train_1000.parquet')

In [ ]:
exp_clf = setup(data = train, target = 'num', fold_shuffle=True, session_id=123)

In [ ]:
best = compare_models(sort='r2', n_select=3)

In [ ]:
lgbm = create_model('lightgbm')
tuned_lgbm = tune_model(lgbm, n_iter = 1000, optimize = 'MAE', choose_better = True)

In [ ]:
# # 모델 생성하는 법
# lgbm = create_model('lightgbm')

# # 하이퍼 파라미터 튜닝 하는 법
# tuned_lgbm = tune_model(lgbm, n_iter = 1000, optimize = 'R2', choose_better = True)
# tuned_lgbm

# # 모델 생성하는 법
# br = create_model('br')

# # 하이퍼 파라미터 튜닝 하는 법
# tuned_br = tune_model(br, n_iter = 100, optimize = 'R2', choose_better = True)
# tuned_br

# # 모델 블랜딩 하는 법
# blender_3 = blend_models([tuned_lgbm, tuned_br])
# blender_3

# # 모델 앙상블 하는 법
# esemble_lgbm = ensemble_model(lgbm, fold = 5, n_estimators = 100, choose_better = True)
# esemble_lgbm
# esemble_lgbm_tuned = ensemble_model(tuned_lgbm, fold = 10, n_estimators = 100, choose_better = True)
# esemble_lgbm_tuned
# esemble_blender_3 = ensemble_model(blender_3, fold = 10, n_estimators = 100, choose_better = True)
# esemble_blender_3

# 최종 모델
# final_model = finalize_model(esemble_lgbm_tuned)
# final_model

# predict 하는 법
# prediction = predict_model(final_model, data = x_test)

# y_pred = prediction['Label']

# MAE =  mean_absolute_error(y_test, y_pred)
# MSE = mean_squared_error(y_test, y_pred)
# RMSE = np.sqrt(MSE)
# MSLE = mean_squared_log_error(y_test, y_pred)
# RMSLE = np.sqrt(mean_squared_log_error(y_test, y_pred))
# R2 = r2_score(y_test, y_pred)

# print(MAE, MSE, RMSE, MSLE, RMSLE, R2, sep='\n')

# fig = plt.figure(figsize=(30,10))
# fig.add_subplot()

# plt.plot(y_pred[:200].tolist(), label='Prediction', color='red')
# plt.plot(y_test[:200].tolist(), label='Test', color='green')
# plt.legend(fontsize = 30)